In [45]:
import pandas as pd
import numpy as np
df = pd.read_csv('./Kurla_Station_data_Sheet1 - Kurla_Station_data_Sheet1.csv')
df

,From Date,To Date,PM2.5,PM10,NO,NO2,SO2,CO,Ozone,RH,WS,WD,AT
0,17-03-2021 00:00,17-03-2021 01:00,137.96,None,None,129.99,73.27,2.32,3.47,44.77,0.82,236.97,25.74
1,17-03-2021 01:00,17-03-2021 02:00,111.31,None,None,None,None,None,None,None,None,None,None
2,17-03-2021 02:00,17-03-2021 03:00,49,None,None,None,None,None,None,None,None,None,None
3,17-03-2021 03:00,17-03-2021 04:00,32.83,None,7.97,22.28,None,None,None,61.28,0.9,184.03,25.09
4,17-03-2021 04:00,17-03-2021 05:00,50.19,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17559,18-03-2023 15:00,18-03-2023 16:00,14.93,217.88,10.18,25.75,25.55,0.33,49.06,62.99,2.59,231.42,32.5
17560,18-03-2023 16:00,18-03-2023 17:00,17.07,250.22,12.9,24.92,25.54,0.26,44.69,66.23,2.86,242.13,31.92
17561,18-03-2023 17:00,18-03-2023 18:00,12.36,236.92,10.55,22.7,25.56,0.23,48.02,70.65,2.54,243.53,31.17
17562,18-03-2023 18:00,18-03-2023 19:00,13.25,189.68,10.98,24.55,25.55,0.25,43.67,77.6,2.05,244.63,30.36


In [46]:
df.replace('None', np.nan, inplace=True)
filled=df.fillna(method='ffill')
filled=df.fillna(method='bfill')
filled.drop(filled.tail(1).index,inplace=True)
filled

,From Date,To Date,PM2.5,PM10,NO,NO2,SO2,CO,Ozone,RH,WS,WD,AT
0,17-03-2021 00:00,17-03-2021 01:00,137.96,272,7.97,129.99,73.27,2.32,3.47,44.77,0.82,236.97,25.74
1,17-03-2021 01:00,17-03-2021 02:00,111.31,272,7.97,22.28,42.73,0.26,3.95,61.28,0.9,184.03,25.09
2,17-03-2021 02:00,17-03-2021 03:00,49,272,7.97,22.28,42.73,0.26,3.95,61.28,0.9,184.03,25.09
3,17-03-2021 03:00,17-03-2021 04:00,32.83,272,7.97,22.28,42.73,0.26,3.95,61.28,0.9,184.03,25.09
4,17-03-2021 04:00,17-03-2021 05:00,50.19,272,27.11,61.81,42.73,0.26,3.95,60.11,0.73,119.47,24.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17558,18-03-2023 14:00,18-03-2023 15:00,15.71,267.75,9.77,27.88,25.53,0.34,53.17,64.27,2.52,233.19,32.33
17559,18-03-2023 15:00,18-03-2023 16:00,14.93,217.88,10.18,25.75,25.55,0.33,49.06,62.99,2.59,231.42,32.5
17560,18-03-2023 16:00,18-03-2023 17:00,17.07,250.22,12.9,24.92,25.54,0.26,44.69,66.23,2.86,242.13,31.92
17561,18-03-2023 17:00,18-03-2023 18:00,12.36,236.92,10.55,22.7,25.56,0.23,48.02,70.65,2.54,243.53,31.17


In [47]:
filled[['PM2.5','PM10','NO','NO2','SO2','CO','Ozone','RH','WS','WD','AT']]=filled[['PM2.5','PM10','NO','NO2','SO2','CO','Ozone','RH','WS','WD','AT']].astype(float)
filled1=filled
filled1.drop(filled1.columns[[0,1]], axis=1, inplace=True)
filled1=filled1.apply(pd.to_numeric, errors='coerce')

filled1['PM10'].dtype
filled1.dtypes

PM2.5    float64
PM10     float64
NO       float64
NO2      float64
SO2      float64
CO       float64
Ozone    float64
RH       float64
WS       float64
WD       float64
AT       float64
dtype: object

In [58]:
def determine_outlier_thresholds_iqr(dataframe, col_name, th1=0.25, th3=0.75):
    quartile1 = dataframe[col_name].quantile(th1)
    quartile3 = dataframe[col_name].quantile(th3)
    iqr = quartile3 - quartile1
    upper_limit = quartile3 + 1.5 * iqr
    lower_limit = quartile1 - 1.5 * iqr
    return lower_limit, upper_limit

def check_outliers_iqr(dataframe, col_name):
    lower_limit, upper_limit = determine_outlier_thresholds_iqr(dataframe, col_name)
    if dataframe[(dataframe[col_name] > upper_limit) | (dataframe[col_name] < lower_limit)].any(axis=None):
        return True
    else: 
        return False

def replace_with_thresholds_iqr(dataframe,cols, th1=0.05, th3=0.95, replace=False):
    from tabulate import tabulate
    data = []
    for col_name in cols:
        if col_name != 'Outcome':
            outliers_ = check_outliers_iqr(dataframe,col_name)
            count = None
            lower_limit, upper_limit = determine_outlier_thresholds_iqr(dataframe, col_name, th1, th3)
            if outliers_:
                count = dataframe[(dataframe[col_name] > upper_limit) | (dataframe[col_name] < lower_limit)][cols].count()
                if replace: 
                    if lower_limit < 0:
                        # We don't want to replace with negative values, right!
                        dataframe.loc[(dataframe[col_name] > upper_limit), col_name] = upper_limit
                    else:
                        dataframe.loc[(dataframe[col_name] < lower_limit), col_name] = lower_limit
                        dataframe.loc[(dataframe[col_name] > upper_limit), col_name] = upper_limit
            outliers_status = check_outliers_iqr(dataframe, col_name)
            data.append([outliers_, outliers_status, count, col_name, lower_limit, upper_limit ])
    table = tabulate(data, headers=['Outliers (Previously)', 'Outliers', 'Count', 'Column', 'Lower Limit', 'Upper Limit'], tablefmt='rst', numalign='right')
    print("Removing Outliers using IQR")
    print(table)

replace_with_thresholds_iqr(filled1, filled1.columns)
# check_outliers_iqr(filled1, 'PM2.5')

Removing Outliers using IQR
=======================  ==========  ============  ========  =============  =============
Outliers (Previously)    Outliers    Count         Column      Lower Limit    Upper Limit
=======================  ==========  ============  ========  =============  =============
True                     True        PM2.5    305  PM2.5          -158.265        289.031
                                     PM10     305
                                     NO       305
                                     NO2      305
                                     SO2      305
                                     CO       305
                                     Ozone    305
                                     RH       305
                                     WS       305
                                     WD       305
                                     AT       305
                                     dtype: int64
True                     True        PM2.5    31   PM10       

In [60]:
determine_outlier_thresholds_iqr(filled1, 'PM2.5')

(-36.207499999999996, 111.01249999999999)